# Pandas

This will provide a tutorial on performing data analysis in Python. This assumes that you are already familiar with Python, including the numpy and matplotlib libraries. The `pandas` library will be introduced and used here.

In [2]:
# Import all libraries needed for the tutorial
import matplotlib.pyplot as plt
import pandas as pd
import random

# This is only required to enable inline plotting in Jupyter notebooks
%matplotlib inline

We'll be using Pandas primarily to be able to load and manipulate data in Excel sheets. First, however, we'll need to go through some basics. We'll do so by going through an example where we create, manipulate and analyse a data set.

### Create Data

The data set will consist of 1,000 baby names and the number of births recorded for the year 1880. We will also add plenty of duplicates so you will see the same baby name more than once. You can think of the multiple entries per name simply being different hospitals around the country reporting the number of births per baby name. So if two hospitals reported the baby name "Bob", the data will have two values for the name Bob. We will start by creating the random set of baby names.

In [3]:
# The inital set of baby names
names = ['Bob','Jessica','Mary','John','Mel']

To make a random list of 1000 baby names using the five above we will generate a random number between 0 and 4. To do this we will be using the functions `seed`, `randint`, `len`, `range`, and `zip`.

In [4]:
random.seed(500)
random_names = []

for i in range(1000):
    random_number = random.randint(0, len(names) - 1)
    random_names.append(names[random_number])

# The number of births per name for the year 1880
births = [random.randint(0, 1000) for i in range(1000)]

# Merge the names and the births data set using the zip function
BabyDataSet = list(zip(random_names,births))

print(BabyDataSet[:10])

[('John', 478), ('Mel', 875), ('Mary', 306), ('Jessica', 942), ('John', 422), ('Jessica', 607), ('Mel', 752), ('Jessica', 424), ('Mary', 950), ('John', 552)]



* `seed(500)` - Create seed
* `randint(0, len(names))` - Generate a random integer between zero and the length of the list `names`. We need to subtract this by one since `randint` will also include the maximum value
* `names[n]` - Select the name where its index is equal to n.
* `for i in range(n)` - Loop until i is equal to n, i.e. 1,2,3,....n.

We are basically done creating the data set. We will now use the pandas library to export this data set into a CSV file.

In [5]:
df = pd.DataFrame(data=BabyDataSet, columns=['Names', 'Births'])

df[:10]

Names  Births
0     John     478
1      Mel     875
2     Mary     306
3  Jessica     942
4     John     422
5  Jessica     607
6      Mel     752
7  Jessica     424
8     Mary     950
9     John     552

`df` is a `DataFrame` object. You can think of this object holding the contents of the BabyDataSet in a format similar to an excel spreadsheet. When printing dataframes they are automatically tabulated for easier reading.

We will now export dataframes to a CSV file, which we will name `births1880.csv`. The function `to_csv` will be used to export the file. The only parameters we will use is `index` and `header`. Setting these parameters to `False` will prevent the index and header names from being exported. Change the value of these parameters to get a better undertanding of their use. 

In [7]:
df.to_csv('births1880.csv', index=False, header=False)

### Grab Data

The read back the CSV file we will use the function `read_csv`. We will simply pass it the filename of the text file we want to read.

In [8]:
df = pd.read_csv('births1880.csv')

df[:10]

John  478
0      Mel  875
1     Mary  306
2  Jessica  942
3     John  422
4  Jessica  607
5      Mel  752
6  Jessica  424
7     Mary  950
8     John  552
9      Bob  567

Note that the `read_csv` function treated the first record in the CSV file as the header names. This is obviously not correct since the text file did not provide us with header names. The correct this we will pass the `header` parameter to the `read_csv` function and set it to None

In [9]:
df = pd.read_csv('births1880.csv', header=None)

df[:10]

0    1
0     John  478
1      Mel  875
2     Mary  306
3  Jessica  942
4     John  422
5  Jessica  607
6      Mel  752
7  Jessica  424
8     Mary  950
9     John  552

If we wanted to give the columns specific names, we would have to pass another parameter called `names`. We can also omit the header parameter in this case

In [10]:
df = pd.read_csv('births1880.csv', names=['Names', 'Births'])

df[:10]

Names  Births
0     John     478
1      Mel     875
2     Mary     306
3  Jessica     942
4     John     422
5  Jessica     607
6      Mel     752
7  Jessica     424
8     Mary     950
9     John     552

You can think of the numbers [0, 1, 2, ...] as the row numbers in an Excel file. In pandas these are part of the *index* of the dataframe. `[Names, Births]` can be though of as column headers similar to the ones found in an Excel spreadsheet.

We can also print out some information about the read in dataframe

In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
Names     1000 non-null object
Births    1000 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


This tells us that:
* There are 1000 records in the data set
* There is a column named `Names` with 1000 values
* There is a column named `Births` with 1000 values
* Out of the two columns, one is numeric and the other is non numeric

We can use the `head()` function which by default will return the first five record. You can also pass in a number `n` to return the top `n` records of the dataframe. On the other hand, `tail()` returns the last five records.

In [213]:
df.head()

Names  Births
0     John     478
1      Mel     875
2     Mary     306
3  Jessica     942
4     John     422

In [214]:
df.tail()

Names  Births
995  John     479
996   Bob     501
997   Mel     438
998  John     707
999   Bob     259

In [215]:
# Delete the csv file now, we are done using it
import os
os.remove('births1880.csv')

### Prepare Data

The data we have consists of baby names and the number of births in the year 1880. We already know that we have 1,000 records and none of the records are missing. We can verify the `Names` collumn still only has five unique names. We can use the `unique` property of the dataframe to find all the unique records of the `Names` column

In [216]:
# Method 1:
df['Names'].unique()

array(['John', 'Mel', 'Mary', 'Jessica', 'Bob'], dtype=object)

In [217]:
# Method 2:
df['Names'].describe()

count        1000
unique          5
top       Jessica
freq          214
Name: Names, dtype: object

Since we have multiple values per baby name, we need to aggregate this data so we only have a baby name appear once. This means the 1000 rows will need to become 5. We can accomplish this by using the `groupby` function.

In [12]:
# Create a groupby object
name = df.groupby('Names')

# Apply the sum function to the grouby object
df = name.sum()

# Show result
df

Births
Names          
Bob       97002
Jessica  107717
John      99585
Mary      96137
Mel      101127

### Analyze Data

To find the most popular name or the baby name with the highest birth rate we can do one of the following:
* Sort the dataframe and select the top row
* Use the `max()` attribute to find the maximum value

In [219]:
# Method 1:
Sorted = df.sort_values(['Births'], ascending=False)
Sorted.head(1)

Births
Names          
Jessica  107717

In [220]:
# Method 2
df['Births'].max()

107717

### Present Data

Here we can plot the Births column and label the graph to show the highest point on the graph. We will use the `plot.bar` function to generate a bar plot of the data contained in a data frame, as well as present a table contained the births per name sorted in ascending order using the `sort_values` function

In [221]:
df['Births'].plot.bar()
df['Births'].plot.line()

df.sort_values(by='Births', ascending=False)

Births
Names          
Jessica  107717
Mel      101127
John      99585
Bob       97002
Mary      96137

You can find a list of tutorial on Pandas [here](http://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html)

## Useful features of Pandas

### Export data to Excel

Note that for Pandas to be able to write and read Excel files you need to:
* Have Microsoft Excel installed
* Install the xlrd and openpyxl Python packages (if you are using Anaconda these should be installed automatically)

In [13]:
# Create DataFrame
d = [1, 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9]
df = pd.DataFrame(d, columns = ['Number'])

df

Number
0       1
1       2
2       3
3       4
4       5
5       6
6       7
7       8
8       9

In [223]:
# Export to Excel
df.to_excel('pandas.xlsx', sheet_name='my_sheet', index=False)

Here we first create a pandas dataframe which consists of a single columns containing numbers. Then we use the `to_excel` function to create an Excel file containing a single worksheet called `my_sheet`. This sheet then contains the data, excluding the index (otherwise two columns would be created, one for the index and one for the numbers).

### Import from Excel

In [224]:
# Read back the excel file generated in the previous example
df = pd.read_excel('pandas.xlsx', 0)
df.head()

Number
0       1
1       2
2       3
3       4
4       5

The above reads back the Excel file generated in the previous example. The second argument is the sheet name or ID, which in this case is 0, meaning that we're reading the first sheet in the file. 